In [1]:
import fitz

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

print(extract_text_from_pdf("test.pdf"))


# calculate sha256 file hash
import hashlib

def sha256_checksum(filename, block_size=65536):

    sha256 = hashlib.sha256()

    with open(filename, 'rb') as f:
        for block in iter(lambda: f.read(block_size), b''):
            sha256.update(block)

    return sha256.hexdigest()

filehash = sha256_checksum("test.pdf")



 
Planning Committee 7 November 2023 
 
 
 
173 
MINUTES 
 
OF A MEETING OF THE  
 
PLANNING COMMITTEE 
 
 
held on 7 November 2023 
Present: 
 
Cllr L Morales (Chairman) 
Cllr T Aziz (Vice-Chair) 
 
Cllr G Cosnahan 
Cllr S Dorsett 
Cllr S Greentree 
Cllr D Jordan 
 
Cllr C Martin 
Cllr S Mukherjee 
Cllr S Oades 
Cllr T Spenser 
 
 
Also Present: Councillors  A Javaid, I Johnson, L Lyons and E Nicholson.  
 
1. 
APOLOGIES FOR ABSENCE  
 
No apologies for absence were received. 
2. 
DECLARATIONS OF INTEREST  
 
In accordance with the Members’ Code of Conduct, Councillor G Cosnahan declared a 
non-pecuniary interest in minute item 6b. 2023/0505 Woking Community Hospital, Woking 
– arising from the Councillors’ spouse being a Senior Manager and Nurse for the Inpatients 
Department of St Peters Hospital. As this role was within the same NHS trust, but not the 
same hospital, as the application site, the Councillor did not have a disclosable pecuniary 
interest. The interest was such that s

In [2]:
import fsspec
import adlfs

storage_options = {
    "connection_string": "DefaultEndpointsProtocol=https;AccountName=bbstdamaskdevfrce001;AccountKey=7kOfCx3VNSF3Z8uVwANsXSJD4M08EAInBt7cr4KQP4M4o1+AMAd3KzsQurCvkWyIQo5KvoeKTKTn+AStFxnlQQ==;BlobEndpoint=https://bbstdamaskdevfrce001.blob.core.windows.net/;QueueEndpoint=https://bbstdamaskdevfrce001.queue.core.windows.net/;TableEndpoint=https://bbstdamaskdevfrce001.table.core.windows.net/;FileEndpoint=https://bbstdamaskdevfrce001.file.core.windows.net/;"
}

fs = adlfs.AzureBlobFileSystem(**storage_options)

#create container if not exists called data
fs.mkdir("data")


In [3]:
import time

# write file text to blob as hash.txt
with fs.open("data/" + filehash + ".txt", "w") as f:
    f.write(extract_text_from_pdf("test.pdf"))


In [4]:
filehash2 = sha256_checksum("essay.txt")

#read in PaulGrahamEssay.txt
read_file = open("PaulGrahamEssay.txt", "r")
essay = read_file.read()

with fs.open("data/" + filehash2 + ".txt", "w") as f:
    f.write(essay)

In [5]:
starttime = time.time()
# use fast range seek on blob to retrieve 100-200

readchunks = [
    {"start": 10, "end": 200},
    {"start": 20, "end": 300},
    {"start": 30, "end": 400},
    {"start": 40, "end": 500},
    {"start": 50, "end": 600},
    {"start": 60, "end": 700},
    {"start": 70, "end": 800},
    {"start": 80, "end": 900},
    {"start": 90, "end": 100},
    {"start": 100, "end": 110},
    {"start": 110, "end": 120},
    {"start": 120, "end": 130},
    {"start": 130, "end": 140},
    {"start": 140, "end": 150},
    {"start": 150, "end": 160},
    {"start": 160, "end": 170},
    {"start": 170, "end": 180},
    {"start": 180, "end": 190},
    {"start": 190, "end": 200},
]

#shuffle chunks
import random
#random.shuffle(readchunks)

with fs.open("data/" + filehash + ".txt", "rb") as f:
    for chunk in readchunks:
        f.seek(chunk["start"])
        #print(f.read(chunk["end"] - chunk["start"]))
    
with fs.open("data/" + filehash2 + ".txt", "rb") as f:
    for chunk in readchunks:
        f.seek(chunk["start"])
        print(f.read(chunk["end"] - chunk["start"]))
    

print("time taken: ", time.time() - starttime)

b"ked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to"
b"bruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot,"
b"1\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried "
b'college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning

In [15]:
from azure.cosmos import CosmosClient, PartitionKey, exceptions, TransactionalBatch

# Initialize Cosmos Client
url = "https://bb-cosmos-damask-dev-efc-001.documents.azure.com:443/"
key = "zp9TjjX12kOrflpO5Qy1kbcLjEFcTcX8fkvnEriF0YnBuILBaIFupOIMTj8NtXFppAkJGAR7XfJ6ACDbSHTo9g=="
client = CosmosClient(url, credential=key)

# Create a new database
database_name = "testDatabase"
try:
    database = client.create_database(database_name)
except exceptions.CosmosResourceExistsError:
    database = client.get_database_client(database_name)

# Create a new container
container_name = "testContainer"
try:
    container = database.create_container(
        id=container_name, partition_key=PartitionKey(path="/id")
    )
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(container_name)

# Assuming container and partition_key_value are defined
partition_key_value = "partitionKey"

batch = TransactionalBatch(container)

for i in range(1, 10000):
    batch.create_item(
        {"id": "item{0}".format(i), "value": i, "partitionKey": partition_key_value}
    )

response_list = batch.execute()

ImportError: cannot import name 'TransactionalBatch' from 'azure.cosmos' (/Users/mikerenwick/Documents/GitHub/damask/venv/lib/python3.11/site-packages/azure/cosmos/__init__.py)